In [1]:
from atproto import AsyncClient 
from dotenv import dotenv_values 
import asyncio 
import numpy as np
import networkx as nx 
import matplotlib.pyplot as plt 
import json
import atproto_client.models as models
from dateutil.parser import isoparse
import datetime 
from datetime import timezone
import pandas as pd 

dotenv = dotenv_values('.env')

client = AsyncClient() 
await client.login(dotenv['USERNAME'], dotenv['PASSWORD']);

In [4]:
async def get_account_data(tag, limit=10):
    response = await client.app.bsky.feed.search_posts(
        params=models.AppBskyFeedSearchPosts.Params(
            q=tag,  
            limit=limit,
            sort='top'
        )
    )

    authors = set([post.author.handle for post in response.posts])
    posts_by_author = {
        author: [post for post in response.posts if post.author.handle == author]
        for author in authors
    }
    tasks = [make_metrics(author, posts) for author, posts in posts_by_author.items()]
    results = await asyncio.gather(*tasks)
    df = pd.DataFrame(dict(results)).transpose()
    df['Engagement Score'] = (((df['Total Engagements'] / df['Number of posts']) * df['Followers']) / [age.seconds for age in df['Age at First Post']])
    return df.sort_values(by=['Engagement Score'], ascending=False)

    

async def make_metrics(author, posts):
    metrics = {}
    metrics["Followers"] = len((await client.get_followers(author)).followers)
    metrics["Following"] = len((await client.get_follows(author)).follows)
    metrics['Number of posts'] = len(posts)
    metrics['Total Likes'] = sum([post.like_count for post in posts])
    metrics['Total Replies'] = sum([post.reply_count for post in posts])
    metrics['Total Quotes'] = sum([post.quote_count for post in posts])
    metrics['Age at First Post'] = min([isoparse(post.record.created_at) for post in posts]) - isoparse(posts[0].author.created_at)
    metrics['Total Engagements'] = metrics['Total Likes'] +  metrics['Total Replies'] + metrics['Total Quotes'] 
    return author, metrics

metrics = await get_account_data("#germany", limit=100)
metrics

,Followers,Following,Number of posts,Total Likes,Total Replies,Total Quotes,Age at First Post,Total Engagements,Engagement Score
kilianschoenberger.bsky.social,49,33,3,617,8,3,"26 days, 2:51:17.565000",628,0.998086
gripennews.bsky.social,50,50,1,40,0,1,"382 days, 1:23:33.102000",41,0.408937
jsgamok.bsky.social,49,50,1,101,8,4,"26 days, 5:08:05.044000",113,0.29954
superchinois801.bsky.social,50,2,1,313,18,0,"30 days, 16:01:51.522000",331,0.286774
aktmodels.bsky.social,49,50,2,318,16,9,"87 days, 9:50:23.603000",343,0.237233
...,...,...,...,...,...,...,...,...,...
justhanan.com,49,50,1,13,0,0,"93 days, 20:01:42.335000",13,0.008835
mattyraatzsch.bsky.social,36,45,1,16,0,0,"97 days, 19:29:59.002000",16,0.008205
cl1ck1.bsky.social,26,50,1,13,0,0,"439 days, 22:04:45.476000",13,0.004252
visiblethings.bsky.social,25,28,1,10,0,0,"12 days, 21:03:42.489000",10,0.003297
